In [ ]:
# Packages

import os
import gc
import polars as pl
import numpy as np 
import pandas as pd
import json
import glob
import pickle
import matplotlib.pyplot as plt
import xgboost as xgb
import joblib
import mlflow
import mlflow.xgboost
import warnings
from itertools import product
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Loading and concatenating the whole dataset

class LoadData:
    
    def __init__(self, file_paths):
        self.file_paths = file_paths
        
    def load_and_concat(self):

        partitioned_data = [pl.read_parquet(file_path) for file_path in self.file_paths]
        df = pl.concat(partitioned_data, rechunk=False)
        
        return df
    
# Specify file paths
file_paths = sorted(glob.glob('Data/train.parquet/*/*.parquet'))

# Initialize the loader and load data as a lazy frame
loader = LoadData(file_paths)
df_full = loader.load_and_concat()

In [ ]:
'''
# Collect unique date_ids

unique_date_ids = df_full.select('date_id').unique().sort('date_id')
total_days = unique_date_ids.height
trading_days_per_year = 252  # Approximate number of trading days per year
total_years = total_days / trading_days_per_year
train_ratio = 0.9
train_days = int(total_days * train_ratio)
holdout_days = total_days - train_days
training_date_ids = unique_date_ids['date_id'][:train_days]
holdout_date_ids = unique_date_ids['date_id'][train_days:]

# Training Data (Cross-Validation)
train_df_collected = df_full.filter(pl.col('date_id').is_in(training_date_ids))

# Hold-Out Data
holdout_df_collected = df_full.filter(pl.col('date_id').is_in(holdout_date_ids))

print(f"Total number of unique days: {total_days}")
'''

In [ ]:
'''
df_full = df_full.sort(['date_id', 'time_id'])

# Get the indices where each date_id starts
date_id_series = df_full['date_id'].to_numpy()
unique_date_ids = np.unique(date_id_series)
date_id_to_indices = {}

for date_id in unique_date_ids:
    indices = np.where(date_id_series == date_id)[0]
    date_id_to_indices[date_id] = indices

# Define the number of folds
n_splits = 8

# Calculate the number of days per fold
days_per_fold = len(unique_date_ids) // n_splits

fold_indices = []
for i in range(n_splits):
    start_day = i * days_per_fold
    if i == n_splits - 1:
        # Include all remaining days in the last fold
        end_day = len(unique_date_ids)
    else:
        end_day = (i + 1) * days_per_fold
    fold_date_ids = unique_date_ids[start_day:end_day]
    fold_indices.append(np.concatenate([date_id_to_indices[date_id] for date_id in fold_date_ids]))
'''

In [ ]:
# Import necessary libraries
warnings.filterwarnings('ignore')

# Set up MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Time Series Forecasting Experiment 2")
df_full = df_full.sort(['date_id', 'time_id'])

# Define hyperparameter distributions for random search
from scipy.stats import uniform, randint

param_distributions = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.19),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5),
    'min_child_weight': randint(1, 6),
    'reg_alpha': uniform(0.0, 1.0),
    'reg_lambda': uniform(0.5, 4.5)
}

# Number of hyperparameter combinations to try
n_iter = 50

# Initialize random number generator
rng = np.random.default_rng(seed=42)

# Generate random hyperparameter combinations
hyperparameter_list = []
for _ in range(n_iter):
    params = {
        'n_estimators': rng.integers(50, 500),
        'max_depth': rng.integers(3, 10),
        'learning_rate': rng.uniform(0.01, 0.2),
        'subsample': rng.uniform(0.6, 1.0),
        'colsample_bytree': rng.uniform(0.6, 1.0),
        'gamma': rng.uniform(0, 0.5),
        'min_child_weight': rng.integers(1, 6),
        'reg_alpha': rng.uniform(0.0, 1.0),
        'reg_lambda': rng.uniform(0.5, 5.0)
    }
    hyperparameter_list.append(params)


# Exclude non-informative features
excluded_features = ['responder_6']

# Define feature columns (excluding the target and unwanted features)
feature_cols = [col for col in df_full.columns if col not in excluded_features]

# Define custom cross-validation folds
n_splits = 8
data_length = len(df_full)

# Split the data into n_splits folds
fold_sizes = np.full(n_splits, data_length // n_splits)
fold_sizes[:data_length % n_splits] += 1  # Distribute the remainder

indices = np.arange(data_length)
current = 0
fold_indices_list = []
for fold_size in fold_sizes:
    start, stop = current, current + fold_size
    fold_indices_list.append(indices[start:stop])
    current = stop

# Create custom fold indices
validation_fraction = 0.2

custom_fold_indices = []

for i in range(1, n_splits):
    # Get indices for folds i - 1 and i
    fold_i_minus1_indices = fold_indices_list[i - 1]
    fold_i_indices = fold_indices_list[i]

    # Split fold_i_indices into training and validation sets
    val_size = int(len(fold_i_indices) * validation_fraction)
    if val_size == 0:
        val_size = 1  # Ensure at least one sample in validation

    train_fold_i_indices = fold_i_indices[:-val_size]
    val_fold_i_indices = fold_i_indices[-val_size:]

    # Combine training indices
    train_indices = np.concatenate([fold_i_minus1_indices, train_fold_i_indices])

    # Validation indices
    val_indices = val_fold_i_indices

    custom_fold_indices.append((train_indices, val_indices))

def train_evaluate_model(df_full, feature_cols, params, custom_fold_indices):
    import matplotlib.pyplot as plt  # Ensure matplotlib is imported
    import pandas as pd             # Import pandas for DataFrame operations
    import os                       # For file operations if needed

    fold_rmse_list = []
    fold_mse_list = []
    fold_r2_list = []

    for fold, (train_indices, val_indices) in enumerate(custom_fold_indices):
        print(f"\nProcessing Fold {fold + 1}")
        try:
            # Select training and validation data from Polars DataFrame
            df_train = df_full[train_indices]
            df_val = df_full[val_indices]

            # Convert Polars DataFrames to Pandas DataFrames for XGBoost
            df_train_pd = df_train.to_pandas()
            df_val_pd = df_val.to_pandas()

            # Extract features and target
            X_train = df_train_pd[feature_cols].astype(np.float32)
            y_train = df_train_pd['responder_6'].astype(np.float32)
            X_val = df_val_pd[feature_cols].astype(np.float32)
            y_val = df_val_pd['responder_6'].astype(np.float32)

            # Convert to XGBoost DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Set up model parameters
            model_params = params.copy()
            model_params.update({
                'objective': 'reg:squarederror',
                'tree_method': 'hist',     # Efficient tree method
                'eval_metric': 'rmse',
                'n_jobs': -1,              # Utilize all CPU cores
                'verbosity': 0,
                'seed': 42                 # For reproducibility
            })

            # Specify evaluation set
            evals = [(dval, 'validation')]

            # Train the model with early stopping
            model = xgb.train(
                model_params,
                dtrain,
                num_boost_round=1000,
                evals=evals,
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict and evaluate
            y_pred = model.predict(dval)
            mse = mean_squared_error(y_val, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_val, y_pred)

            print(f"Fold {fold + 1} - RMSE: {rmse:.4f}, MSE: {mse:.4f}, R^2: {r2:.4f}")

            # Store metrics
            fold_rmse_list.append(rmse)
            fold_mse_list.append(mse)
            fold_r2_list.append(r2)

            # Log metrics
            mlflow.log_metric(f"rmse_fold_{fold + 1}", rmse)
            mlflow.log_metric(f"mse_fold_{fold + 1}", mse)
            mlflow.log_metric(f"r2_fold_{fold + 1}", r2)

            # --------- Feature Importance Code Starts Here ---------

            # Extract feature importance
            feature_importance = model.get_score(importance_type='gain')
            sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
            importance_df = pd.DataFrame(sorted_importance, columns=['Feature', 'Importance'])

            # Limit to top 15 features
            top_n = 15
            importance_df = importance_df.head(top_n)

            # Save feature importance to a CSV file
            importance_csv = f'feature_importance_fold_{fold + 1}.csv'
            importance_df.to_csv(importance_csv, index=False)

            # Log the CSV file as an artifact
            mlflow.log_artifact(importance_csv, artifact_path=f'feature_importance/fold_{fold + 1}')

            # Plot feature importance
            plt.figure(figsize=(10, 6))
            importance_df.plot(kind='bar', x='Feature', y='Importance', legend=False)
            plt.title(f'Feature Importance - Fold {fold + 1}')
            plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels
            plt.tight_layout()

            # Save the plot as an image
            importance_plot = f'feature_importance_fold_{fold + 1}.png'
            plt.savefig(importance_plot)

            # Log the image as an artifact
            mlflow.log_artifact(importance_plot, artifact_path=f'feature_importance/fold_{fold + 1}')

            # Close the plot to free memory
            plt.close()

            # --------- Feature Importance Code Ends Here ---------

            # Clean up
            del df_train, df_val, df_train_pd, df_val_pd, X_train, X_val, y_train, y_val, dtrain, dval, model
            gc.collect()

        except Exception as e:
            print(f"Error on fold {fold + 1}: {e}")
            mlflow.log_param(f"error_fold_{fold + 1}", str(e))
            continue

    # Calculate average metrics
    avg_rmse = np.mean(fold_rmse_list)
    std_rmse = np.std(fold_rmse_list)
    avg_mse = np.mean(fold_mse_list)
    std_mse = np.std(fold_mse_list)
    avg_r2 = np.mean(fold_r2_list)
    std_r2 = np.std(fold_r2_list)

    return {
        'avg_rmse': avg_rmse,
        'std_rmse': std_rmse,
        'avg_mse': avg_mse,
        'std_mse': std_mse,
        'avg_r2': avg_r2,
        'std_r2': std_r2
    }

# Initialize results list
grid_search_results = []

# Begin MLflow run
with mlflow.start_run(run_name="Random Hyperparameter Search with Custom Cross-Validation") as parent_run:
    for idx, params in enumerate(hyperparameter_list):
        print(f"\nEvaluating hyperparameters set {idx + 1}/{n_iter}: {params}")

        # Start a nested MLflow run for this hyperparameter combination
        with mlflow.start_run(run_name=f"Params set {idx + 1}", nested=True):
            # Log hyperparameters
            mlflow.log_params(params)

            # Train and evaluate the model
            metrics = train_evaluate_model(df_full, feature_cols, params, custom_fold_indices)

            # Log average metrics
            mlflow.log_metric("avg_rmse", metrics['avg_rmse'])
            mlflow.log_metric("std_rmse", metrics['std_rmse'])
            mlflow.log_metric("avg_mse", metrics['avg_mse'])
            mlflow.log_metric("std_mse", metrics['std_mse'])
            mlflow.log_metric("avg_r2", metrics['avg_r2'])
            mlflow.log_metric("std_r2", metrics['std_r2'])

            # Save results
            result = {
                'params': params,
                'avg_rmse': metrics['avg_rmse'],
                'std_rmse': metrics['std_rmse'],
                'avg_mse': metrics['avg_mse'],
                'std_mse': metrics['std_mse'],
                'avg_r2': metrics['avg_r2'],
                'std_r2': metrics['std_r2']
            }
            grid_search_results.append(result)

            # End of hyperparameter combination run
            # The 'with' block takes care of ending the run

    # Save grid search results as an artifact

    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, (np.integer, np.int64, np.int32)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float64, np.float32)):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.bool_)):
                return bool(obj)
            else:
                return super(NumpyEncoder, self).default(obj)

    # At the end of your script
    with open('grid_search_results.json', 'w') as f:
        json.dump(grid_search_results, f, indent=4, cls=NumpyEncoder)
    mlflow.log_artifact('grid_search_results.json')


In [ ]:
# Initialize the model with adjusted hyperparameters
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=10,         # Increased from 5 to 10
    max_depth=6,
    learning_rate=0.1,       # Increased from 0.01 to 0.1
    reg_alpha=0.1,           # L1 regularization
    reg_lambda=1.0,          # L2 regularization
    random_state=42,
    verbosity=1
)

# Define chunk size
chunk_size = 1000000  # Increased from 50,000 to 1,000,000

# Placeholder for the existing booster
existing_model = None

# Placeholder for feature columns
feature_columns = None

# Optional: Shuffle the dataset before training to ensure representative chunks
# Uncomment the following lines if your dataset isn't already shuffled
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Starting incremental training...")
for i in tqdm(range(0, len_train, chunk_size), desc="Training Progress", leave=True):
    # Load and process a chunk of data
    chunk = train_df.slice(i, chunk_size).collect().to_pandas()
    X_chunk = chunk.drop(columns=['responder_6'])
    y_chunk = chunk['responder_6']
    
    # Save feature columns from the first chunk
    if i == 0:
        feature_columns = X_chunk.columns.tolist()
        with open("feature_columns.pkl", "wb") as f:
            pickle.dump(feature_columns, f)
    
    # Ensure feature alignment
    X_chunk = X_chunk[feature_columns]
    
    # Define a small validation set from the current chunk
    # Here, we take 10% of the chunk as a temporary validation set
    X_val = X_chunk.sample(frac=0.1, random_state=42)
    y_val = y_chunk.loc[X_val.index]
    
    # Remove validation samples from training data
    X_train_chunk = X_chunk.drop(index=X_val.index)
    y_train_chunk = y_chunk.drop(index=X_val.index)
    
    # Fit the model incrementally with early stopping
    xgb_model.fit(
        X_train_chunk, 
        y_train_chunk, 
        xgb_model=existing_model, 
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # Update the existing booster
    existing_model = xgb_model.get_booster()
    
    # Monitor total number of trees
    total_trees = xgb_model.get_booster().trees_to_dataframe().shape[0]
    print(f"Total number of trees after this chunk: {total_trees}")
    
    # Sample prediction for monitoring
    sample_pred = xgb_model.predict(X_train_chunk.sample(5))
    print(f"Sample Predictions after this chunk: {sample_pred}")
    
    # Stop training if total trees exceed 10,000 (adjust as needed)
    if total_trees >= 100000:
        print("Reached the maximum number of trees. Stopping training.")
        break

print("Training complete.")

# Save the entire model using joblib
joblib.dump(xgb_model, "xgb_baseline_model.joblib")
print("Model saved using joblib.")